In [31]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
device = ('cuda' if torch.cuda.is_available() else 'cpu');
device

'cuda'

In [33]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/fashion-mnist.csv");
df.shape

(10000, 785)

In [34]:
X_train,X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:,0], test_size=0.2, random_state=42);
print(len(X_train));

8000


In [35]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [43]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features.values, dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels.values, dtype=torch.long);

  def __len__(self):
    return len(self.features);

  def __getitem__(self, index):
    return self.features[index], self.labels[index];

In [44]:
train_dataset = CustomDataset(X_train, y_train);
test_dataset = CustomDataset(X_test, y_test);

In [45]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,pin_memory=True)

In [47]:
class MyNN(nn.Module):
  def __init__(self, input_channels):
    super().__init__();

    self.features = nn.Sequential(
        nn.Conv2d(input_channels, 32, kernel_size=3, padding="same"),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(32, 64, kernel_size=3, padding="same"),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7, 128),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(p=0.4),
        nn.Linear(64,10)
    )

  def forward(self, features):
    x = self.features(features);
    x = self.classifier(x);
    return x

In [40]:
epochs = 100
learning_rate = 0.001;


In [49]:
model = MyNN(1)
model.to(device);
criterion = nn.CrossEntropyLoss();
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4);

In [50]:
for epoch in range(epochs):
  total_epoch_loss = 0;
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    output = model(batch_features);
    loss = criterion(output, batch_labels)
    optimizer.zero_grad();
    loss.backward();
    optimizer.step();
    total_epoch_loss += loss.item();

  avg_epoch_loss = total_epoch_loss/len(train_loader);
  if epoch % 10 == 0:
    print(
          "Epoch: {}/{}... ".format(epoch, epochs),
          "Loss: {:.6f}...".format(avg_epoch_loss),
        )

Epoch: 0/100...  Loss: 2.168212...
Epoch: 10/100...  Loss: 0.862879...
Epoch: 20/100...  Loss: 0.665744...
Epoch: 30/100...  Loss: 0.583819...
Epoch: 40/100...  Loss: 0.519021...
Epoch: 50/100...  Loss: 0.456632...
Epoch: 60/100...  Loss: 0.414393...
Epoch: 70/100...  Loss: 0.378830...
Epoch: 80/100...  Loss: 0.357849...
Epoch: 90/100...  Loss: 0.330178...


In [51]:
model.eval()

MyNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [53]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    outputs = model(batch_features);
    _, predicted = torch.max(outputs,1);
    total += batch_labels.shape[0];
    correct += (predicted == batch_labels).sum().item();
  print(f"Accuracy of the convolutional neural network on the {X_test.shape[0]} test images: %d %%" % (100 * correct / total))

Accuracy of the convolutional neural network on the 2000 test images: 88 %


In [55]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    outputs = model(batch_features);
    _, predicted = torch.max(outputs,1);
    total += batch_labels.shape[0];
    correct += (predicted == batch_labels).sum().item();
  print(f"Accuracy of the convolutional neural network on the {X_train.shape[0]} test images: %d %%" % (100 * correct / total))

Accuracy of the convolutional neural network on the 8000 test images: 93 %
